In [ ]:
# Benodigde packages

import json            # Package om .json files in te laden (bvb kolomnamen zijn zo opgeslagen)
import getpass         # Package om een paswoordveldje te genereren.
import mysql.connector # MySQL package
import numpy as np
import os
import pandas as pd    # Populaire package voor data-verwerking
import sys
from basics import*

sys.version_info       # Check python versie, wij veronderstellen 3.6

In [ ]:
def verbind_met_GB(username, hostname, gegevensbanknaam):
    """
    Maak verbinding met een externe gegevensbank
    
    :param  username:          username van de gebruiker, string
    :param  hostname:          naam van de host, string.
                               Dit is in het geval van een lokale server gewoon 'localhost'
    :param  gegevensbanknaam:  naam van de gegevensbank, string.
    :return connection:        connection object, dit is wat teruggeven wordt 
                               door connect() methods van packages die voldoen aan de DB-API
    """
    
    password = getpass.getpass() # Genereer vakje voor wachtwoord in te geven
    
    connection = mysql.connector.connect(host=hostname,
                                         user=username,
                                         passwd=password,
                                         db=gegevensbanknaam)
    return connection


def run_query(connection, query):
    """
    Voer een query uit op een reeds gemaakte connectie, geeft het resultaat van de query terug
    """
    
    # Making a cursor and executing the query
    cursor = connection.cursor()
    cursor.execute(query)
    
    # Collecting the result and casting it in a pd.DataFrame
    res = cursor.fetchall()
    
    return res


def res_to_df(query_result, column_names):
    """
    Giet het resultaat van een uitgevoerde query in een 'pandas dataframe'
    met vooraf gespecifieerde kolomnamen.
    
    Let op: Het resultaat van de query moet dus exact evenveel kolommen bevatten
    als kolomnamen die je meegeeft. Als dit niet het geval is, is dit een indicatie
    dat je oplossing fout is. (Gezien wij de kolomnamen van de oplossing al cadeau doen)
    
    """
    df = pd.DataFrame(query_result, columns=column_names)
    return df

In [ ]:
# Dictionary van kolomnamen inladen

filename = os.path.join(os.getcwd(), 'solution', 'all_q_colnam.json')
col_names = json.load(open(filename, 'r'))    

col_names # Inspecteer dictionary

In [ ]:
def query_01(connection, column_names):
    # Bouw je query
    query="""
    SELECT   t.name, t.yearID, t.HR
    FROM     Teams as t
    ORDER BY t.HR DESC;
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_01'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_01(c, col_names['query_01'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 

In [ ]:
def query_02(connection, column_names, datum = '1980-01-16'):
    # Bouw je query
    query="""
    SELECT   Master.nameFirst, Master.nameLast, 
             Master.birthYear, Master.birthMonth, Master.birthDay
    FROM     Master
    WHERE    Master.debut> {}
    ORDER BY Master.nameLast;
    """.format(datum)
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_02'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_02(c, col_names['query_02'], datum = '1980-01-16')

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 

In [ ]:
def query_03(connection, column_names):
    # Bouw je query
    query="""
    SELECT DISTINCT MS.nameFirst, MS.nameLast, t.name
    FROM     Teams as t, Managers as MG, Master as MS
    WHERE    MG.teamID = t.teamID AND MG.yearID = t.yearID
             AND MG.plyrMgr = 'N'
             AND MS.playerID = MG.playerID
    
    ORDER BY t.name;
    """
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_03'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_03(c, col_names['query_03'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head()

In [ ]:
def query_04(connection, column_names, datum_x='1980-01-01', datum_y='1980-01-01'):
    # Bouw je query
    query="""
    SELECT DISTINCT t.name, t.Rank, t.W, t.L, MS.nameFirst, MS.nameLast
    FROM   Teams as t, Managers as MG, Master as MS, Halloffame as HF
    WHERE  MS.playerID = MG.playerID AND HF.playerID = MG.playerID
           AND MG.teamID = t.teamID AND MG.yearID  = t.yearID
           AND HF.yearid > {}
           AND MG.yearID > {}
    ORDER by t.name, t.Rank;
    """.format(datum_x, datum_y)

    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_04'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_04(c, col_names['query_04'], datum_x='1980-01-01', datum_y='1980-01-01')

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df


In [ ]:
def query_05(connection, column_names):
    # Bouw je query
    query="""
    SELECT DISTINCT t.name
    FROM   Teams as t
    WHERE  (SELECT  COUNT(*)
            FROM    Teams as t, Managers as MG
            WHERE   t.yearID>'1980-01-01'
                    AND MG.teamID = T.teamID AND MG.plyrMgr = 'Y')>=1;
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_05'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_05(c, col_names['query_05'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

In [ ]:
def query_06(connection, column_names, salaris=20000):
    # Bouw je query
    query="""
    SELECT    t.name, t.yearID, t.Rank, t.W, t.L
    FROM      Teams as t, Salaries as S
    WHERE     t.teamID = S.teamID
    GROUP BY  S.teamID
    
    HAVING    MIN(salary)> {}
    
    ORDER BY  t.W
    """.format(salaris)
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_06'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_06(c, col_names['query_06'], salaris=20000)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

In [ ]:
def query_07(connection, column_names):
    # Bouw je query
    query="""
    SELECT DISTINCT MS.nameLast, MS.nameFirst
    FROM  Managers as MG, Master as MS, Awardsmanagers as AM
    
    WHERE MG.playerID = MS.playerID AND MS.playerID = AM.playerID
    GROUP BY AM.playerID
    HAVING COUNT(DISTINCT AM.awardID)=2

    ORDER by MS.nameLast;
    """
    #    AND   AM.awardID LIKE '%SN%'  
    # AND   AM.awardID LIKE '%BW%' 

    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_07'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_07(c, col_names['query_07'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df


In [ ]:
def query_08(connection, column_names, jaar=1990, lengte=75):
    # Bouw je query
    query="""
    
    
    SELECT   MS.birthState, AVG(MS.weight), AVG(MS.height),
             AVG(B.HR), AVG(P.SV)
    FROM     Master as MS, Batting as B, Pitching as P,
             Halloffame as HF
                
    WHERE    HF.yearID >= {} AND
             MS.playerID = HF.playerID AND
             MS.playerID = B.playerID AND
             MS.playerID = P.playerID
             
    GROUP BY MS.BirthState
    HAVING AVG(MS.Height)> {}
    
   
    
        
    ORDER BY MS.BirthState;
    """.format(jaar,lengte)
                 
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_08'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_08(c, col_names['query_08'], jaar=1990, lengte=75)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

In [ ]:
def query_09(connection, column_names, jaar=1975):
    # Bouw je query
    query="""
    SELECT   t.yearID, t.name, t.HR
    FROM     Teams as t
    WHERE    t.yearID = {}
    ORDER BY t.HR DESC
    LIMIT    1,1; 
    
    
     """.format(jaar)
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_09'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_09(c, col_names['query_09'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

In [ ]:
def query_10(connection, column_names, jaar=1990):
    # Bouw je query
    
    query="""
    SELECT t.yearID, t.name, t.Rank, t.G
    FROM Teams as t
    WHERE t.yearID = {} and exists( SELECT ap.playerID
                                    FROM   Appearances as ap, AwardsPlayers as a
                                    WHERE  t.teamID = ap.teamID AND {} = ap.yearID 
                                           AND ap.playerID = a.playerID
    GROUP BY a.playerID
    HAVING count(a.awardID)= 1)
    """.format(jaar, jaar)
    
    
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_10'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_10(c, col_names['query_10'], jaar=1990)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 